In [10]:
import os
openai_api_key = os.environ['OPENAI_API_KEY']

from datasets import Dataset 
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())


# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from langchain.llms.base import LLM

# openai_model = ChatOpenAI(
#     model_name="gpt-4",
#     openai_api_key=API_KEY,
#     max_tokens=2048
# )

# ragas_openai_model = LLM(openai_model)

# openai_embeddings = OpenAIEmbeddings(
#     model="text-embedding-ada-002",
#     openai_api_key=API_KEY
# )


from ragas import evaluate
# 參考的檔案跟AI回應關，數值介於0~1之間，數值越高，代表回答得越好
# 這邊的範例是問超級盃的問題，回答的內容是超級盃的資訊，所以faithfulness的分數應該是很高的
# 但是第二題的context和answer不符合，所以faithfulness的分數應該是很低的
data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts' : [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'], 
    ['The Green Bay Packers...Green Bay, Wisconsin.','The Packers compete...Football Conference']],
}

dataset = Dataset.from_dict(data_samples)


In [12]:

from ragas.metrics import (faithfulness, answer_relevancy)
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity
# metrics = [faithfulness, answer_relevancy]

metrics = [
    LLMContextRecall(llm=evaluator_llm), 
    FactualCorrectness(llm=evaluator_llm), 
    Faithfulness(llm=evaluator_llm),
    SemanticSimilarity(embeddings=evaluator_embeddings)
]

# Evaluate
result = evaluate(
    dataset, metrics=metrics, llm=evaluator_llm, embeddings=evaluator_embeddings
)

df = result.to_pandas()
print(df)

ValueError: The metric [context_recall] that is used requires the following additional columns ['reference'] to be present in the dataset.